### Load the Dataset and import libraries:

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('mushrooms.csv')
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [3]:
le = LabelEncoder()
df_new = df.apply(le.fit_transform)
df_new.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [4]:
data = df_new.values
print(data.shape)
print(type(data))
print(data[:5,:])

(8124, 23)
<class 'numpy.ndarray'>
[[1 5 2 4 1 6 1 0 1 4 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 9 1 0 1 0 0 4 0 2 2 2 7 7 0 2 1 4 3 2 1]
 [0 0 2 8 1 3 1 0 0 5 0 2 2 2 7 7 0 2 1 4 3 2 3]
 [1 5 3 8 1 6 1 0 1 5 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 3 0 5 1 1 0 4 1 3 2 2 7 7 0 2 1 0 3 0 1]]


In [5]:
data_X = data[:,1:]
data_Y = data[:,0]

### Devide data into test and train:

In [6]:
x_train,x_test,y_train,y_test = train_test_split(data_X,data_Y,test_size=0.2)

In [7]:
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(6499, 22) (1625, 22)
(6499,) (1625,)


In [8]:
np.unique(y_train)

array([0, 1])

### Note:
To convert the numeric data back to non numeric,we can use the inverse tranform method
of sklearn by importing from sklearn.preprocessing.StandardScaler

## Build the Classifier:

In [10]:
# 1st compute the prior probab i.e P(Y = 1) or P(Y = c) for a given class c

def prior_prob(y_train,label):
    total_examples = y_train.shape[0]
    class_examples = np.sum(y_train==label)
    
    return class_examples / float(total_examples)

In [11]:
def conditional_prob(x_train,y_train,feature_name,feature_val,label):
    x_filtered = x_train[y_train == label]
   
    numerator = np.sum(x_filtered[:,feature_name] == feature_val)
    
    denominator = np.sum(y_train == label)
    return numerator/float(denominator)



## Find the Posterior Probability and make Predictions:

In [12]:
def predict(x_train,y_train,XTest):
    no_of_classes = np.unique(y_train)
    no_of_features = x_train.shape[1]
    
    posterior_probs = []
    
    for label in no_of_classes:
        likelihood = 1.0
        for feature in range(no_of_features):
            cond_prob = conditional_prob(x_train,y_train,feature,XTest[feature],label)
            likelihood *= cond_prob
            
        prior = prior_prob(y_train,label)
        current_class_post_prob = likelihood*prior
        posterior_probs.append(current_class_post_prob)
    
    prediction = np.argmax(posterior_probs)
    return prediction

In [13]:
output = predict(x_train,y_train,x_test[1])
print(output)

1


In [14]:
# And the actual class is:
print(y_test[1])

1


## Score Method:

In [17]:
# we'll compute a list of perdictions for all x_test points and compare them with y_test to
# measure the accuracy of our model
def score(x_train,y_train,x_test,y_test):
    pred = []
    
    for i in range(x_test.shape[0]):
        predicted_labels = predict(x_train,y_train,x_test[i])
        pred.append(predicted_labels)
    
    # to compare directly,make pred as a numpy array:
    pred = np.array(pred)
    accuracy = np.sum(pred==y_test)/y_test.shape[0]
    return accuracy

In [18]:
print(score(x_train,y_train,x_test,y_test))

0.9969230769230769


Which is a very good accuracy value for a binary classifier!